<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/KERC_1022_%EC%9D%B4%EB%A6%84%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **configurations**

### Memory cleaner

In [1]:
memory_clean = True 
#memory_clean = False 

import gc
import torch

if memory_clean: 
  gc.collect()
  torch.cuda.empty_cache()


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
num_kfold = 5 #
NUM_EPOCH = 3
TRAIN_BATCH = 64
EVAL_BATCH = 16
#num_kfold = 7 #
test_private = True

#submission_csv length
if test_private:
  test_num = 4950
else:
  test_num = 2566

#서로 다른 BERT 앙상블 5개 할 것 -> 모델 history는 주석처리를 하여 남겨둘 것 

model_name = "klue/roberta-large" #사용할 모델 1
#model_name = "klue/bert-base" #사용할 모델 1
#model_name = "snunlp/KR-ELECTRA-discriminator" #사용할 모델 2
#model_name = "kykim/bert-kor-base" #사용할 모델 3: colab pro 기준 2epoch: 2~3분 x 5폴드
#model_name = "beomi/KcELECTRA-base" #사용할 모델 4
#model_name = "lighthouse/mdeberta-v3-base-kor-further" #사용할 모델 5


DATE = '10-21'

#submission_comment = f"{DATE} using KLUE-BERT" #최종 저장되는 submission.csv의 이름이 됨
submission_comment = f"{DATE} using KLUE-ROBERTA-L" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using KRELECTRA" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using BERT-Kor-base" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using KcELECTRA-base" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using mDeBERTa-kor" #최종 저장되는 submission.csv의 이름이 됨

#model_nickname = "KLUE-BERT"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
model_nickname = "KLUE-ROBERTA-L"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "KR-ELECTRA"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "BERT-Kor-base"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "kc-electra-base"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "mDeberta-kor"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨


## install

In [5]:
!pip install transformers
!pip install datasets #from huggingface
!pip install evaluate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 81.0 MB/s 
     |████████████████████████████████| 163 kB 88.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 93.1 MB/s 
     |████████████████████████████████| 115 kB 88.6 MB/s 
     |████████████████████████████████| 127 kB 90.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [6]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [7]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop

import numpy as np 

train_pair['context2'] = train_pair['context'].fillna(' ')
train_pair['sentence2'] = train_pair['sentence'].fillna(' ')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair

train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(i==1):
    scenario = scenario +"\""+ df.iloc[i-1]['sentence2']+"\"" + "라고 " +df.iloc[i-1]['person']+"가 말하자, "+df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(i==2):
    scenario = scenario + df.iloc[i-2]['person'] + "가 " + "\""+ df.iloc[i-2]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-1]['person'] + "가 " + df.iloc[i-1]['sentence2']+"라고 말하자, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(2<i and i <= len(df)-1):
    scenario = scenario + df.iloc[i-3]['person'] + "가 " + "\""+ df.iloc[i-3]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-2]['person'] + "가 " + df.iloc[i-2]['sentence2']+"라고 말하자, "+ df.iloc[i-1]['person'] + "가 " + "\""+ df.iloc[i-1]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario


def trans_df(train_pair, train):
  for i in scene_df[:]:
    temp = train_pair[train_pair['scene']==i].reset_index(drop=True)
    train+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df(train_pair, train)
train = pd.DataFrame(train, columns=('scenario',))
print(train.iloc[5]['scenario'])

train['label'] = train_pair['label']
train
train_pairs = train

labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }



train_pairs['scenario'] = train_pairs['scenario'].str.replace('순경', '아버지')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('과자', '어머니')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('건강', '장남')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('현찰', '둘째 아들')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('이상', '셋째 아들')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('청난', '여자 주인공')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('범인', '어영의 아버지')
train_pairs['scenario'] = train_pairs['scenario'].str.replace('솔이', '가정부')

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대 <sep> 지구대가 "김경사님, 아직 안가셨어요? 시간 다됐을텐데."라고 말하고, 순경가 근무중인데 어딜가?라고 말하자, 지구대가 "다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요."라고 말하고, 순경가 "그럴거 없어." 라고 말하였다.


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=1.0, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [9]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
import pandas as pd 
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling,AutoModelForMaskedLM
import evaluate


max_input_length = 200
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = AutoTokenizer.from_pretrained(model_name)


accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    #metric = evaluate.load('glue', 'qnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)
    return metric.compute(predictions=predictions, references=labels, average='micro')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)
#klue/roberta-large

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'cl

In [10]:
from transformers import EarlyStoppingCallback
#Gather your training arguments~
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    do_train=True,
    do_eval=True,
#    per_device_train_batch_size=32,
#    per_device_eval_batch_size=32,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
#    warmup_steps=1000,
    weight_decay=0.01,
)


trainer = Trainer(
        model=model,
        args=training_args,
        #train_dataset = train_dataset,
        #eval_dataset= val_dataset,
        data_collator=data_collator, 
        tokenizer=tokenizer,  
        compute_metrics = compute_metrics,
        #model_init=model_init,  
    )

#Focal Loss
class CustomLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
          labels = inputs.get("labels")
          # forward pass
          outputs = model(**inputs)
          logits = outputs.get("logits")  
          # compute custom loss (suppose one has 3 labels with different weights)
          loss_fct = FocalLoss().cuda()
          loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
          return (loss, outputs) if return_outputs else loss


def preprocess_function(examples):
    inputs = [doc for doc in examples["scenario"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    return model_inputs


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(i==1):
    scenario = scenario +"\""+ df.iloc[i-1]['sentence2']+"\"" + "라고 " +df.iloc[i-1]['person']+"가 말하자, "+df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(i==2):
    scenario = scenario + df.iloc[i-2]['person'] + "가 " + "\""+ df.iloc[i-2]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-1]['person'] + "가 " + df.iloc[i-1]['sentence2']+"라고 말하자, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(2<i and i <= len(df)-1):
    scenario = scenario + df.iloc[i-3]['person'] + "가 " + "\""+ df.iloc[i-3]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-2]['person'] + "가 " + df.iloc[i-2]['sentence2']+"라고 말하자, "+ df.iloc[i-1]['person'] + "가 " + "\""+ df.iloc[i-1]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario


def trans_df2(test_pair, test):
  for i in scene_df2[:]:
    temp = test_pair[test_pair['scene']==i].reset_index(drop=True)
    test+=[mk_scenario(j, temp) for j in range(len(temp))]



k_fold_logits = []

#num_kfold = 10
#kfold = StratifiedKFold(n_splits=5, random_state=77, shuffle=True)

kfold = StratifiedKFold(n_splits=5, random_state=77, shuffle=True)
#kfold = StratifiedKFold(n_splits=5)

for train_df_idx, val_df_idx in kfold.split(train_pairs, train_pairs['label']):

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

    trainer = Trainer(
    #trainer = CustomLossTrainer(
            model=model,
            args=training_args,
            #train_dataset = train_dataset,
            #eval_dataset= val_dataset,
            data_collator=data_collator, 
            tokenizer=tokenizer,  
            compute_metrics = compute_metrics,
            #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]  
        )
    

    train_df = train_pairs.iloc[train_df_idx]
    val_df = train_pairs.iloc[val_df_idx]

  
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    train_dataset =train_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])
    val_dataset = val_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])

    #Collect your model, training args, dataset, data collator, tokenizer in Trainer
    trainer.train_dataset = train_dataset
    trainer.eval_dataset = val_dataset
    

    #best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")    
    trainer.train()



    data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"

    test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')
    test_pair = test_data[['sentence_id','context','sentence','person', 'scene']]
    
    #PRIVATE
    if test_private:
      test_data_private = pd.read_csv(data_path+"/private_test_data.tsv", sep='\t')
      test_pair_private = test_data_private[['sentence_id','context','sentence','person', 'scene']]
      test_pair = pd.concat([test_pair,test_pair_private],ignore_index=True)
    test_pair['context2'] = test_pair['context'].fillna(' ')
    test_pair['sentence2'] = test_pair['sentence'].fillna(' ')
    #모든 글자 합치기
    test = list()
    scene_df2 = test_pair['scene']
    scene_df2 = scene_df2.drop_duplicates().reset_index(drop=True)
        
    trans_df2(test_pair, test)
    test = pd.DataFrame(test, columns=('scenario',))
    #print("--------------------\n", test)

    #test_pairs = test_pair[['label','new']]
    test['sentence_id'] = test_pair['sentence_id']

    
    test['scenario'] = test['scenario'].str.replace('순경', '아버지')
    test['scenario'] = test['scenario'].str.replace('과자', '어머니')
    test['scenario'] = test['scenario'].str.replace('건강', '장남')
    test['scenario'] = test['scenario'].str.replace('현찰', '둘째 아들')
    test['scenario'] = test['scenario'].str.replace('이상', '셋째 아들')
    test['scenario'] = test['scenario'].str.replace('청난', '여자 주인공')
    test['scenario'] = test['scenario'].str.replace('범인', '어영의 아버지')
    test['scenario'] = test['scenario'].str.replace('솔이', '가정부')

    test_dataset = Dataset.from_pandas(test)
    test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['scenario'])
    

    predictions = trainer.predict(test_dataset)

    k_fold_logits.append(predictions.predictions)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5871
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1835
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.514517,0.782016
2,0.561500,0.480463,0.836512
3,0.300200,0.481800,0.846049
4,0.300200,0.794750,0.844005
5,0.131600,0.867825,0.841962


...

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5871
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1835


Epoch,Training Loss,Validation Loss,F1
1,No log,0.458741,0.817439


***** Running Evaluation *****
  Num examples = 1468
  Batch size = 16


KeyboardInterrupt: ignored

In [ ]:

#predict= k_fold_logits[0] + k_fold_logits[1] + k_fold_logits[2] +k_fold_logits[3] +k_fold_logits[4]
#predict= k_fold_logits[0] + k_fold_logits[1] + k_fold_logits[2] +k_fold_logits[3] +k_fold_logits[4]



for i in range(num_kfold):
  if i==0:
    predict = k_fold_logits[i]
  else:
    predict += k_fold_logits[i]


#predict = k_fold_logits[0]

list1 = []
list2 = []


for i in range(test_num):
  list1.append(i)
  list2.append(-1)



raw_data = {'Id': list1,
            'Predicted': list2
            }
df = pd.DataFrame(raw_data)



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

df['Predicted'] = np.argmax(predict, axis=-1)

d = ['euphoria','neutral','dysphoria']

df['Predicted'] = [d[label] for label in df['Predicted']]


df['Id'] = test_dataset['sentence_id']

#x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

df



df.to_csv(f'drive/MyDrive/호반우NLP_한국인감정/submission_{model_nickname}_{num_kfold}FOLD_{submission_comment}_batch_{TRAIN_BATCH}_{EVAL_BATCH}_epoch_{NUM_EPOCH}.csv', index=False)

df

In [ ]:
k_fold_logits[0]

In [ ]:
df['Predicted'].value_counts()


In [ ]:
df['Predicted'][:2566].value_counts()

In [ ]:
#df['Predicted'].to_csv('drive/MyDrive/호반우NLP_한국인감정/'+'df_predicted_kluerobertaL_wd0.07.txt')

### Lastly, save predict(np) file for the Ensemble!

In [ ]:
import numpy as np 

ensemble_save =True
day = '1023'
nick = '정석'
if ensemble_save == True:
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}_kfold_0', k_fold_logits[0]) #predict_{model_nickname}.npy
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}_kfold_1', k_fold_logits[1]) #predict_{model_nickname}.npy
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}_kfold_2', k_fold_logits[2]) #predict_{model_nickname}.npy
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}_kfold_3', k_fold_logits[3]) #predict_{model_nickname}.npy
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}_kfold_4', k_fold_logits[4]) #predict_{model_nickname}.npy
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/{day}_{nick}_predict_{model_nickname}', predict) #predict_{model_nickname}.npy

## If you wanna get a ensemble results csv, make run_ensemble =  'True'

In [ ]:
#run_ensemble = True
run_ensemble = False

submission_comment_ensemble = '10-21 ensemble'
#run_ensemble = False


In [ ]:
if run_ensemble:
  predict_ensemble = np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_mDeberta-kor.npy') *0.83
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_kc-electra-base.npy') *0.83
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_BERT-Kor-base.npy') *0.80
  #predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_KR-ELECTRA.npy') *0.80
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_KLUE-ROBERTA-L.npy') *0.85

  raw_data = {'Id': list1,
              'Predicted': list2
              }
  df = pd.DataFrame(raw_data)

  labels = {'euphoria':0,
            'neutral':1,
            'dysphoria':2
            }

  df['Predicted'] = np.argmax(predict_ensemble, axis=-1)

  d = ['euphoria','neutral','dysphoria']

  df['Predicted'] = [d[label] for label in df['Predicted']]


  df['Id'] = test_dataset['sentence_id']

  #x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

  df.to_csv(f'drive/MyDrive/호반우NLP_한국인감정/submission_ensemble_{submission_comment_ensemble}.csv', index=False)
  df['Predicted'].value_counts()
